# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [1]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 472 (delta 211), reused 159 (delta 159), pack-reused 223 (from 2)
Receiving objects: 100% (472/472), 243.32 KiB | 383.00 KiB/s, done.
Resolving deltas: 100% (251/251), done.


Install all required dependencies:

In [2]:
!pip install -r /content/mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
 

## Model download

In [ ]:
!pip install huggingface_hub

In [ ]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-v0.3'

In [3]:
# Alternatively, you can download the model from mistral

!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2025-03-11 15:28:18--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 172.67.70.68, 104.26.6.117, 104.26.7.117, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|172.67.70.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  32.4MB/s    in 8m 12s  

2025-03-11 15:36:30 (28.1 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [4]:
!DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [5]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [6]:
%cd /content/

/content


In [7]:
# make a new directory called data
!mkdir -p data

In [8]:
# navigate to this data directory
%cd /content/data

/content/data


In [ ]:
# read data into a pandas dataframe
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.5 MB/s eta 0:00:00


In [10]:
import pandas as pd
import json
from datasets import Dataset

# Path to your JSONL file in Google Drive
file_path = '/content/data/mistral_ready_dataset.jsonl'

# Option 1: If your data is actually a JSONL file
def load_jsonl_to_dataset(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Load your data
messages_data = load_jsonl_to_dataset(file_path)

# Option 2: If you want to convert to DataFrame like in the original code
df = pd.DataFrame(messages_data)

# Option 3: If you want to create a Hugging Face dataset directly
# dataset = Dataset.from_list(messages_data)

In [11]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [12]:
# save data into .jsonl files
df_train.to_json("_train.jsonl", orient="records", lines=True)
df_eval.to_json("_eval.jsonl", orient="records", lines=True)

In [13]:
!ls /content/data

_eval.jsonl  mistral_ready_dataset.jsonl  _train.jsonl


In [14]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [15]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /content/data/_train.jsonl

In [16]:
# eval data looks all good
!python -m utils.reformat_data /content/data/_eval.jsonl

In [21]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


0it [00:00, ?it/s]Validating /content/data/_train.jsonl ...

  0% 0/972 [00:00<?, ?it/s]
100% 972/972 [00:00<00:00, 5485.41it/s]
1it [00:00,  5.61it/s]
No errors! Data is correctly formatted!
Stats for /content/data/_train.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:33:38",
        "data_tokens": 64042,
        "train_tokens": 78643200,
        "epochs": "1227.99",
        "max_steps": 300,
        "data_tokens_per_dataset": {
            "/content/data/_train.jsonl": "64042.0"
        },
        "train_tokens_per_dataset": {
            "/content/data/_train.jsonl": "78643200.0"
        },
        "epochs_per_dataset": {
            "/content/data/_train.jsonl": "1228.0"
        }
    }
}
0it [00:00, ?it/s]Validating /content/data/_eval.jsonl ...

100% 51/51 [00:00<00:00, 5272.86it/s]
1it [00:00, 98.88it/s]
No errors! Data is correctly formatted!


## Start training

In [22]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [28]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 5120
batch_size: 1
num_microbatches: 8
max_steps: 300
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra2"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [29]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [30]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

2025-03-11 16:12:58.770577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741709578.793015   13239 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741709578.799386   13239 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 16:12:58.820587: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
args: TrainArgs(data=DataArgs(data='', shuffle=False, instruct_data='/content/data/_train.jsonl', eval_instruct_data='/conten

## Inference

In [ ]:
!pip install mistral_inference

In [ ]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Machine learning is a subset of artificial intelligence that involves the use of algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of computer science that enables machines to learn and improve from experience without being explicitly programmed. Machine learning algorithms can learn from data and make predictions or decisions based
